In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

from config import weather_api_key

In [2]:
data = 'winery_latlong_data.csv'
latlong_df = pd.read_csv(data, low_memory=False)

In [3]:
latlong_df.dropna(inplace=True)
latlong_df

,Unnamed: 0,winery_search_term,latitude,longitude,number_of_results,status
0,0,St. Julian winery US,42.212251,-85.891713,1,OK
1,1,Sweet Cheeks winery US,43.956670,-123.279135,1,OK
2,2,Kirkland Signature winery US,35.896713,-84.140546,1,OK
3,3,Louis M. Martini winery US,38.491708,-122.452308,1,OK
5,5,Richard Böcking winery Germany,49.947177,7.112136,1,OK
...,...,...,...,...,...,...
8693,8693,Ferguson Crest winery US,33.313613,-96.659946,1,OK
8694,8694,Carmel Hills Winery winery US,43.949742,-103.522590,1,OK
8696,8696,Castello Bonomi winery Italy,45.571648,9.967799,1,OK
8697,8697,Barboursville Vineyards winery US,38.159582,-78.277084,1,OK


In [4]:
# Create lists of latitudes and longitudes
lats = list(latlong_df['latitude'])
lngs = list(latlong_df['longitude'])

wineries = list(latlong_df['winery_search_term'])

In [5]:
# Starting URL for Weather Map API Call.
url = "http://history.openweathermap.org/data/2.5/aggregated/year?appid=" + weather_api_key

In [6]:
def get_weather_data (winery, winery_count):
    # Create endpoint URL with each winery.
    winery_url = url + "&lat=" + str(lats[winery_count]) + "&lon=" + str(lngs[winery_count])
    
    winery_temp_data  = []
    winery_pressure_data = []
    winery_humidity_data =[]
    winery_precip_data = []
    
    # Parse the JSON and retrieve data.
    winery_weather = requests.get(winery_url).json()
    
    for j in range (0, 364):
        temp = winery_weather["result"][j]["temp"]["mean"]
        winery_temp_data.append(temp)
        pressure = winery_weather["result"][j]["pressure"]["mean"]
        winery_pressure_data.append(pressure)
        humidity = winery_weather["result"][j]["humidity"]["mean"]
        winery_humidity_data.append(humidity)
        precip = winery_weather["result"][j]["precipitation"]["mean"]
        winery_precip_data.append(precip)
                
    # calculate averages
    avg_temp = sum(winery_temp_data) / len(winery_temp_data)
    avg_pressure = sum(winery_pressure_data) / len(winery_pressure_data)
    avg_humidity = sum(winery_humidity_data) / len(winery_humidity_data)
    avg_precip = sum(winery_precip_data) / len(winery_precip_data)
    
    return avg_temp, avg_pressure, avg_humidity, avg_precip

##### 3 for loops below were run after we realized our first weather pull didn't get all of the information we needed. We were short on time and didn't get a chance to trouble shoot why the for loops were randomly stopping (i.e., the first one ended at a random winery about 3000 records in) 

In [ ]:
# Create an empty list to hold the weather data.
winery_weather_data = []
output_filename = 'winery_weather_v3.csv'

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
winery_count = 0

# Loop through all the wineries in the list.
for i, winery in enumerate(wineries):

    # Group wineries in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {winery}")
    
    # Add 1 to the record count
    record_count += 1
    
    # Run an API request for each of the wineries.
    try:
        # Parse the JSON and retrieve data.
        avg_temp, avg_pressure, avg_humidity, avg_precip = get_weather_data(winery, winery_count)
               
        # Append the winery information into city_data list.
        winery_weather_data.append({"Winery": wineries[winery_count],
                          "Average Temperature (Kelvin)": avg_temp,
                          "Average Air Pressure (hPa)": avg_pressure,
                          "Average Humidity (%)": avg_humidity,
                          "Average Precipitation (mm)": avg_precip})

    # If an error is experienced, skip the winery.
    except:
        print("Weather not found. Skipping...")
        pass
    
    # Add to winery count
    winery_count += 1
    
    # Print status every 100 addresses
    if len(winery_weather_data) % 100 == 0:
    	print("Completed {} of {} address".format(len(winery_weather_data), len(wineries)))
            
    # Every 500 addresses, save progress to file(in case of a failure so we have something!)
    if len(winery_weather_data) % 500 == 0:
        pd.DataFrame(winery_weather_data).to_csv("{}_bak".format(output_filename))


    # save full results to CSV
    pd.DataFrame(winery_weather_data).to_csv(output_filename)

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | St. Julian winery US
Processing Record 2 of Set 1 | Sweet Cheeks winery US
Processing Record 3 of Set 1 | Kirkland Signature winery US
Processing Record 4 of Set 1 | Louis M. Martini winery US
Processing Record 5 of Set 1 | Richard Böcking winery Germany
Processing Record 6 of Set 1 | Bianchi winery US
Processing Record 7 of Set 1 | Castello di Amorosa winery US
Processing Record 8 of Set 1 | Envolve winery US
Processing Record 9 of Set 1 | Erath winery US
Processing Record 10 of Set 1 | Feudi del Pisciotto winery Italy
Processing Record 11 of Set 1 | Hawkins Cellars winery US
Processing Record 12 of Set 1 | Robert Hall winery US
Processing Record 13 of Set 1 | Sundance winery Chile
Processing Record 14 of Set 1 | Trump winery US
Processing Record 15 of Set 1 | RustRidge winery US
Processing Record 16 of Set 1 | Souverain winery US
Processing Record 17 of Set 1 | Tres Palacios winery Chile
Proces

Processing Record 48 of Set 3 | Flying Cloud winery US
Processing Record 49 of Set 3 | Majolini winery Italy
Processing Record 50 of Set 3 | Patrick Javillier winery France
Processing Record 1 of Set 4 | Rusack winery US
Processing Record 2 of Set 4 | Chehalem winery US
Processing Record 3 of Set 4 | Henri de Villamont winery France
Processing Record 4 of Set 4 | Wittmann winery Germany
Processing Record 5 of Set 4 | Deutz winery France
Processing Record 6 of Set 4 | Domaine Serene winery US
Processing Record 7 of Set 4 | Dr. Loosen winery Germany
Processing Record 8 of Set 4 | Bell winery US
Processing Record 9 of Set 4 | Babcock winery US
Processing Record 10 of Set 4 | Lyeth winery US
Processing Record 11 of Set 4 | Knapp winery US
Processing Record 12 of Set 4 | Lava Cap winery US
Processing Record 13 of Set 4 | Michael Pozzan winery US
Processing Record 14 of Set 4 | Domaine du Grand Cros winery France
Processing Record 15 of Set 4 | El Coto winery Spain
Processing Record 16 of Se

Processing Record 43 of Set 6 | Deerfield Ranch winery US
Processing Record 44 of Set 6 | Efeste winery US
Processing Record 45 of Set 6 | Firesteed winery US
Processing Record 46 of Set 6 | Iron Hub winery US
Processing Record 47 of Set 6 | Kooyong winery Australia
Processing Record 48 of Set 6 | Aiken winery US
Processing Record 49 of Set 6 | Donkey & Goat winery US
Processing Record 50 of Set 6 | Finn Hill winery US
Completed 300 of 7994 address
Processing Record 1 of Set 7 | La Playa winery Chile
Processing Record 2 of Set 7 | Stomping Ground winery US
Processing Record 3 of Set 7 | Terrapura winery Chile
Processing Record 4 of Set 7 | Domaine de Thulon winery France
Processing Record 5 of Set 7 | Kontos winery US
Processing Record 6 of Set 7 | Laurent Gauthier winery France
Processing Record 7 of Set 7 | Leaping Horse winery US
Processing Record 8 of Set 7 | Domaine Bott-Geyl winery France
Processing Record 9 of Set 7 | Domaine Collotte winery France
Processing Record 10 of Set 7 

Processing Record 36 of Set 9 | Domaine de Rome winery France
Processing Record 37 of Set 9 | Domdechant Werner winery Germany
Processing Record 38 of Set 9 | Dr. Fischer winery Germany
Processing Record 39 of Set 9 | Pinord winery Spain
Processing Record 40 of Set 9 | Summers winery US
Processing Record 41 of Set 9 | Viszlay Vineyards winery US
Processing Record 42 of Set 9 | Yealands winery New Zealand
Processing Record 43 of Set 9 | Ampelos winery US
Processing Record 44 of Set 9 | Ascencion winery US
Processing Record 45 of Set 9 | Challen winery US
Processing Record 46 of Set 9 | Crux winery US
Processing Record 47 of Set 9 | Damiani winery US
Processing Record 48 of Set 9 | Domaine Clos Gautier winery France
Processing Record 49 of Set 9 | Domaine Harmonie des Arpents winery France
Processing Record 50 of Set 9 | Doren winery US
Processing Record 1 of Set 10 | 14 Hands winery US
Processing Record 2 of Set 10 | Chateau le Grand Verdus winery France
Processing Record 3 of Set 10 | 

Processing Record 29 of Set 12 | Château Los Boldos winery Chile
Processing Record 30 of Set 12 | Domaine de la Sanglière winery France
Processing Record 31 of Set 12 | Château Donjon de Bruignac winery France
Processing Record 32 of Set 12 | Koenig Vineyards winery US
Processing Record 33 of Set 12 | Krutz winery US
Processing Record 34 of Set 12 | Amavi winery US
Processing Record 35 of Set 12 | Bouchaine winery US
Processing Record 36 of Set 12 | Bucher winery US
Processing Record 37 of Set 12 | Jean-Marc Burgaud winery France
Processing Record 38 of Set 12 | Jewelry Box winery US
Processing Record 39 of Set 12 | Magnolia Court winery US
Processing Record 40 of Set 12 | Red Rock winery US
Processing Record 41 of Set 12 | Real Companhia Velha winery Portugal
Processing Record 42 of Set 12 | Decoy winery US
Processing Record 43 of Set 12 | Domaine du Vissoux winery France
Processing Record 44 of Set 12 | Cairdean Estate winery US
Processing Record 45 of Set 12 | Cardwell Hill winery U

...

Processing Record 12 of Set 52 | Otero Ramos winery Argentina
Processing Record 13 of Set 52 | Bodegas Escorihuela winery Argentina
Processing Record 14 of Set 52 | Château Julien winery US
Processing Record 15 of Set 52 | Domaine Chanson winery France
Processing Record 16 of Set 52 | Domaine Rossignol-Trapet winery France
Processing Record 17 of Set 52 | Nicolas Gaudry winery France
Processing Record 18 of Set 52 | Scroll winery US
Processing Record 19 of Set 52 | Stefania Mezzetti winery Italy
Processing Record 20 of Set 52 | French Creek winery US
Processing Record 21 of Set 52 | Giannoni Fabbri winery Italy
Processing Record 22 of Set 52 | Sparkling Pointe winery US
Processing Record 23 of Set 52 | Alexandria Nicole winery US
Processing Record 24 of Set 52 | Domaine de Jarras winery France
Processing Record 25 of Set 52 | Harmonique winery US
Processing Record 26 of Set 52 | Helwig winery US
Processing Record 27 of Set 52 | Jacob Williams winery US
Processing Record 28 of Set 52 | 

Completed 2700 of 7994 address
Processing Record 1 of Set 55 | Shadowbox winery US
Processing Record 2 of Set 55 | Momo winery New Zealand
Processing Record 3 of Set 55 | Upland Estates winery US
Processing Record 4 of Set 55 | Montezuma winery US
Processing Record 5 of Set 55 | Nice Pairing winery US
Processing Record 6 of Set 55 | Alderbrook winery US
Processing Record 7 of Set 55 | SJ Montigny winery Germany
Processing Record 8 of Set 55 | Timothy Malone winery US
Processing Record 9 of Set 55 | Domaine Fernand Engel winery France
Processing Record 10 of Set 55 | Draxton winery US
Processing Record 11 of Set 55 | Rarecat winery US
Processing Record 12 of Set 55 | Robert John winery US
Processing Record 13 of Set 55 | Love That Red winery US
Processing Record 14 of Set 55 | Terra Bella winery US
Processing Record 15 of Set 55 | DeLille winery US
Processing Record 16 of Set 55 | Denis Jamain winery France
Processing Record 17 of Set 55 | Domaine Philippe Raimbault winery France
Proces

Processing Record 41 of Set 57 | Palaia Vineyards winery US
Processing Record 42 of Set 57 | Pomelo winery US
Processing Record 43 of Set 57 | Producteurs Plaimont winery France
Processing Record 44 of Set 57 | Worthy winery US
Processing Record 45 of Set 57 | Marquis de Goulaine winery France
Processing Record 46 of Set 57 | Naches Heights winery US
Processing Record 47 of Set 57 | VinEcol winery Argentina
Processing Record 48 of Set 57 | Tahoe Ridge winery US
Processing Record 49 of Set 57 | Two Wives winery US
Processing Record 50 of Set 57 | Pianetta winery US
Processing Record 1 of Set 58 | Chimney Rock winery US
Processing Record 2 of Set 58 | Enate winery Spain
Processing Record 3 of Set 58 | EnRoute winery US
Processing Record 4 of Set 58 | Hansen-Lauer winery Germany
Processing Record 5 of Set 58 | Nautilus winery New Zealand
Processing Record 6 of Set 58 | Henry Marionnet winery France
Processing Record 7 of Set 58 | Bachelder winery Canada
Processing Record 8 of Set 58 | Mar

Processing Record 35 of Set 60 | The Divining Rod winery US
Processing Record 36 of Set 60 | Leon Beyer winery France
Processing Record 37 of Set 60 | Malk winery US
Processing Record 38 of Set 60 | Vila winery Argentina
Processing Record 39 of Set 60 | Mosby winery US
Processing Record 40 of Set 60 | Santa Florentina winery Argentina
Processing Record 41 of Set 60 | Warwick winery South Africa
Processing Record 42 of Set 60 | Veramonte winery Chile
Processing Record 43 of Set 60 | Domaine Dominique Gruhier winery France
Processing Record 44 of Set 60 | Girly Girl winery US
Processing Record 45 of Set 60 | Flocchini winery US
Processing Record 46 of Set 60 | Aje winery US
Processing Record 47 of Set 60 | Alexander winery Israel
Processing Record 48 of Set 60 | Stemmler winery US
Processing Record 49 of Set 60 | Plumb Cellars winery US
Processing Record 50 of Set 60 | Château Beaubois winery France
Completed 3000 of 7994 address
Processing Record 1 of Set 61 | Château Bouissel winery Fr

Processing Record 22 of Set 63 | Karl Schaefer winery Germany
Processing Record 23 of Set 63 | Patel winery US
Processing Record 24 of Set 63 | Mony winery Israel
Processing Record 25 of Set 63 | Pascal Jolivet winery France
Processing Record 26 of Set 63 | Clos de la Tech winery US
Processing Record 27 of Set 63 | Spottswoode winery US
Processing Record 28 of Set 63 | Matthias Müller winery Germany
Processing Record 29 of Set 63 | Debonné winery US
Processing Record 30 of Set 63 | Ackerman winery US
Processing Record 31 of Set 63 | Dehlinger winery US
Processing Record 32 of Set 63 | Harper Voit winery US
Processing Record 33 of Set 63 | Michaud winery US
Processing Record 34 of Set 63 | Sidebar winery US
Processing Record 35 of Set 63 | Lamadrid winery Argentina
Processing Record 36 of Set 63 | Domaine Ellevin winery France
Processing Record 37 of Set 63 | Dominio IV winery US
Processing Record 38 of Set 63 | Nieto Senetiner winery Argentina
Processing Record 39 of Set 63 | Riebeek C

In [11]:
wineries[3189]

'Hess winery US'

In [ ]:
# Create an empty list to hold the weather data.
winery_weather_data = []
output_filename = 'winery_weather_startatHess.csv'

#RESET COUNTERS
record_count = 1
set_count = 1

# new for loop to pick up where above left off
for i in range (3188,7994):
    
    winery = wineries[i]
    
 # Group wineries in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {winery}")
    
    # Add 1 to the record count
    record_count += 1
    
    # Run an API request for each of the wineries.
    try:
        # Parse the JSON and retrieve data.
        avg_temp, avg_pressure, avg_humidity, avg_precip = get_weather_data(winery, i)
               
        # Append the winery information into city_data list.
        winery_weather_data.append({"Winery": winery,
                          "Average Temperature (Kelvin)": avg_temp,
                          "Average Air Pressure (hPa)": avg_pressure,
                          "Average Humidity (%)": avg_humidity,
                          "Average Precipitation (mm)": avg_precip})

    # If an error is experienced, skip the winery.
    except:
        print("Weather not found. Skipping...")
        pass
    
    # Print status every 100 addresses
    if len(winery_weather_data) % 100 == 0:
    	print("Completed {} of {} address".format(len(winery_weather_data), len(wineries)))
            
    # Every 500 addresses, save progress to file(in case of a failure so we have something!)
    if len(winery_weather_data) % 500 == 0:
        pd.DataFrame(winery_weather_data).to_csv("{}_bak".format(output_filename))


    # save full results to CSV
    pd.DataFrame(winery_weather_data).to_csv(output_filename)

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | Rodrigue Molyneaux winery US
Processing Record 2 of Set 1 | Hess winery US
Processing Record 3 of Set 1 | Montemar Wines winery US
Processing Record 4 of Set 1 | Koonowla winery Australia
Processing Record 5 of Set 1 | Regusci winery US
Processing Record 6 of Set 1 | Baldacci winery US
Processing Record 7 of Set 1 | oops winery Chile
Processing Record 8 of Set 1 | Biltmore Estate winery US
Processing Record 9 of Set 1 | Cousiño-Macul winery Chile
Processing Record 10 of Set 1 | Middle Ridge winery US
Processing Record 11 of Set 1 | Oak Summit winery US
Processing Record 12 of Set 1 | Domaine Weinbach winery France
Processing Record 1 of Set 2 | Epoch Estate Wines winery US
Processing Record 2 of Set 2 | Benmarl winery US
Processing Record 3 of Set 2 | Château Barbebelle winery France
Processing Record 4 of Set 2 | Château Sainte Roseline winery France
Processing Record 5 of Set 2 | Gunter Triebaumer winery Austria
Processing Record 6 of Set 2 | Martinshof

Processing Record 29 of Set 4 | Château Vitallis winery France
Processing Record 30 of Set 4 | Domaine Corine Perchaud winery France
Processing Record 31 of Set 4 | Pierre Vessigaud winery France
Processing Record 32 of Set 4 | Ellman Family winery US
Processing Record 33 of Set 4 | Weingut Am Stein winery Germany
Processing Record 34 of Set 4 | Le Petit Panier winery France
Processing Record 35 of Set 4 | Neal Family winery US
Processing Record 36 of Set 4 | Plan B winery US
Processing Record 37 of Set 4 | The Williamsburg Winery winery US
Processing Record 38 of Set 4 | Domaine de la Grange winery France
Processing Record 39 of Set 4 | Stephen & Walker winery US
Processing Record 40 of Set 4 | Wolf Family winery US
Processing Record 41 of Set 4 | Anomaly winery US
Processing Record 42 of Set 4 | Lea winery US
Processing Record 43 of Set 4 | McCall winery US
Processing Record 44 of Set 4 | Peace Water winery US
Processing Record 45 of Set 4 | Berger winery Austria
Processing Record 46

Processing Record 21 of Set 7 | Crossroads winery New Zealand
Processing Record 22 of Set 7 | Pavillion Crossing winery US
Processing Record 23 of Set 7 | Brutocao winery US
Processing Record 24 of Set 7 | Breathless winery US
Processing Record 25 of Set 7 | Simon Bize winery France
Processing Record 26 of Set 7 | Jawbone winery New Zealand
Processing Record 27 of Set 7 | Walter Dacon winery US
Processing Record 28 of Set 7 | First Press winery US
Processing Record 29 of Set 7 | Hannah Nicole Vineyards winery US
Processing Record 30 of Set 7 | Mazza Chautauqua Cellars winery US
Processing Record 31 of Set 7 | Entropy Cellars winery US
Processing Record 32 of Set 7 | Wilson Daniels winery US
Processing Record 33 of Set 7 | Mandolin winery US
Processing Record 34 of Set 7 | 6 Under winery US
Processing Record 35 of Set 7 | Domaine des Coltabards winery France
Processing Record 36 of Set 7 | Esterházy winery Austria
Processing Record 37 of Set 7 | Heron winery US
Processing Record 38 of S

Processing Record 15 of Set 10 | Maloney winery US
Processing Record 16 of Set 10 | Intrinsic winery US
Processing Record 17 of Set 10 | Desert Wind winery US
Processing Record 18 of Set 10 | Lazy River winery US
Processing Record 19 of Set 10 | Oregon Trails winery US
Processing Record 20 of Set 10 | Oxford Landing winery Australia
Processing Record 21 of Set 10 | Paringa winery Australia
Processing Record 22 of Set 10 | Vincent Sauvestre winery France
Processing Record 23 of Set 10 | Coldstream Hills winery Australia
Processing Record 24 of Set 10 | Gassier winery France
Processing Record 25 of Set 10 | Catrala winery Chile
Processing Record 26 of Set 10 | Gunderloch winery Germany
Processing Record 27 of Set 10 | Dutcher Crossing winery US
Processing Record 28 of Set 10 | Knipser winery Germany
Processing Record 29 of Set 10 | Bodega Privada winery Argentina
Processing Record 30 of Set 10 | Cicchitti winery Argentina
Processing Record 31 of Set 10 | Domaine Billaud-Simon winery Fran

Processing Record 3 of Set 13 | Fattoria la Castellina winery Italy
Processing Record 4 of Set 13 | Martellotto winery US
Processing Record 5 of Set 13 | Snowden winery US
Processing Record 6 of Set 13 | Milton Park winery Australia
Processing Record 7 of Set 13 | Viña La Rosa winery Chile
Processing Record 8 of Set 13 | Veo winery Chile
Processing Record 9 of Set 13 | Domaine des Cantarelles winery France
Processing Record 10 of Set 13 | Edgebaston winery South Africa
Processing Record 11 of Set 13 | Tyee winery US
Processing Record 12 of Set 13 | Batán de Salas winery Spain
Processing Record 13 of Set 13 | Ladera winery US
Processing Record 14 of Set 13 | Haras de Pirque winery Chile
Processing Record 15 of Set 13 | Gann winery US
Processing Record 16 of Set 13 | Bookwalter winery US
Processing Record 17 of Set 13 | Masciarelli winery Italy
Processing Record 18 of Set 13 | Bruno Paillard winery France
Processing Record 19 of Set 13 | Colin winery France
Processing Record 20 of Set 13

...

Processing Record 18 of Set 68 | Rock & Vine winery US
Processing Record 19 of Set 68 | Domaine Torraccia winery France
Processing Record 20 of Set 68 | Lil Rippa winery New Zealand
Processing Record 21 of Set 68 | Château Larteau winery France
Processing Record 22 of Set 68 | Vogelzang winery US
Processing Record 23 of Set 68 | Señorío de Sarria winery Spain
Processing Record 24 of Set 68 | Bartholomew Park winery US
Processing Record 25 of Set 68 | Ahlgren winery US
Processing Record 26 of Set 68 | Robert Pecota winery US
Processing Record 27 of Set 68 | Conn Valley winery US
Processing Record 28 of Set 68 | Baldetti Alfonso winery Italy
Processing Record 29 of Set 68 | Franklin Tate winery Australia
Processing Record 30 of Set 68 | Voss winery New Zealand
Processing Record 31 of Set 68 | Hedges Family Estate winery US
Processing Record 32 of Set 68 | Rocca di Frassinello winery Italy
Processing Record 33 of Set 68 | Treveri Cellars winery US
Processing Record 34 of Set 68 | Verde So

Processing Record 4 of Set 71 | Domaines de Provence winery France
Processing Record 5 of Set 71 | Château des Arnauds winery France
Processing Record 6 of Set 71 | Lenikus winery Austria
Processing Record 7 of Set 71 | Baiting Hollow Farm Vineyard winery US
Processing Record 8 of Set 71 | Domaine Normand winery France
Processing Record 9 of Set 71 | Casa de Vila Verde winery Portugal
Processing Record 10 of Set 71 | Les Dauphins winery France
Processing Record 11 of Set 71 | Michel Tissot winery France
Processing Record 12 of Set 71 | Château la Moulinière winery France
Processing Record 13 of Set 71 | Laroze de Drouhin winery France
Processing Record 14 of Set 71 | Paralelo winery Mexico
Processing Record 15 of Set 71 | Pegasus Estate winery US
Processing Record 16 of Set 71 | Matteo winery US
Processing Record 17 of Set 71 | Männle winery Chile
Processing Record 18 of Set 71 | Kracher winery Austria
Processing Record 19 of Set 71 | James Charles winery US
Processing Record 20 of Set

Processing Record 41 of Set 73 | Domaine de Lischetto winery France
Processing Record 42 of Set 73 | Quintessa winery US
Processing Record 43 of Set 73 | Weingut Geil winery Germany
Processing Record 44 of Set 73 | Lantieri de Paratico winery Italy
Processing Record 45 of Set 73 | Grant Burge winery Australia
Processing Record 46 of Set 73 | Icaria Creek winery US
Processing Record 47 of Set 73 | Matariki winery New Zealand
Processing Record 48 of Set 73 | Môreson winery South Africa
Processing Record 49 of Set 73 | Chase winery US
Processing Record 50 of Set 73 | O winery US
Processing Record 1 of Set 74 | Terras de Alter winery Portugal
Processing Record 2 of Set 74 | Villicana winery US
Processing Record 3 of Set 74 | Koblevo winery Ukraine
Processing Record 4 of Set 74 | Bel Vino winery US
Processing Record 5 of Set 74 | Protea winery South Africa
Processing Record 6 of Set 74 | Hervé Baudry winery France
Processing Record 7 of Set 74 | Danza del Sol winery US
Processing Record 8 o

Processing Record 29 of Set 76 | Glasshaus winery US
Processing Record 30 of Set 76 | Kershaw winery South Africa
Processing Record 31 of Set 76 | Tero Estates winery US
Processing Record 32 of Set 76 | Harbison Wines winery US
Processing Record 33 of Set 76 | Wallace winery US
Processing Record 34 of Set 76 | Bateaux Cellars winery US
Processing Record 35 of Set 76 | Elkhorn Peak winery US
Processing Record 36 of Set 76 | Ilaria winery US
Processing Record 37 of Set 76 | Weingut Burg Ravensburg winery Germany
Processing Record 38 of Set 76 | Shannon winery South Africa
Processing Record 39 of Set 76 | Folin Cellars winery US
Processing Record 40 of Set 76 | Vereinigte Hospitien winery Germany
Processing Record 41 of Set 76 | Diamond Terrace winery US
Processing Record 42 of Set 76 | Masottina winery Italy
Processing Record 43 of Set 76 | Huber winery Austria
Processing Record 44 of Set 76 | Merritt winery US
Processing Record 45 of Set 76 | Weltevrede winery South Africa
Processing Re

Processing Record 18 of Set 79 | Danzante winery Italy
Processing Record 19 of Set 79 | Enrico Gatti winery Italy
Processing Record 20 of Set 79 | Miradou winery France
Processing Record 21 of Set 79 | New Kent winery US
Processing Record 22 of Set 79 | Narmada winery US
Processing Record 23 of Set 79 | Stable Hill winery Australia
Processing Record 24 of Set 79 | Bodegas Carrau winery Uruguay
Processing Record 25 of Set 79 | Familia Traversa winery Uruguay
Processing Record 26 of Set 79 | Adega Vila Real winery Portugal
Processing Record 27 of Set 79 | Ott & Murphy winery US
Processing Record 28 of Set 79 | Pend d'Oreille winery US


In [7]:
wineries[7077]

"Pend d'Oreille winery US"

In [8]:
# Create an empty list to hold the weather data.
winery_weather_data = []
output_filename = 'winery_weather_startatPend.csv'

#RESET COUNTERS
record_count = 1
set_count = 1

# new for loop to pick up where above left off (again)
for i in range (7077,7993):
    
    winery = wineries[i]
    
 # Group wineries in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {winery}")
    
    # Add 1 to the record count
    record_count += 1
    
    # Run an API request for each of the wineries.
    try:
        # Parse the JSON and retrieve data.
        avg_temp, avg_pressure, avg_humidity, avg_precip = get_weather_data(winery, i)
               
        # Append the winery information into city_data list.
        winery_weather_data.append({"Winery": winery,
                          "Average Temperature (Kelvin)": avg_temp,
                          "Average Air Pressure (hPa)": avg_pressure,
                          "Average Humidity (%)": avg_humidity,
                          "Average Precipitation (mm)": avg_precip})

    # If an error is experienced, skip the winery.
    except:
        print("Weather not found. Skipping...")
        pass
    
    # Print status every 100 addresses
    if len(winery_weather_data) % 100 == 0:
    	print("Completed {} of {} address".format(len(winery_weather_data), len(wineries)))
            
    # Every 500 addresses, save progress to file(in case of a failure so we have something!)
    if len(winery_weather_data) % 500 == 0:
        pd.DataFrame(winery_weather_data).to_csv("{}_bak".format(output_filename))


    # save full results to CSV
    pd.DataFrame(winery_weather_data).to_csv(output_filename)

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | Pend d'Oreille winery US
Processing Record 2 of Set 1 | Sentinel winery New Zealand
Processing Record 3 of Set 1 | Jip Jip Rocks winery Australia
Processing Record 4 of Set 1 | Riparian winery New Zealand
Processing Record 5 of Set 1 | Domaine Franck Millet winery France
Processing Record 6 of Set 1 | Bailarin winery US
Processing Record 7 of Set 1 | Wenzel winery Austria
Processing Record 8 of Set 1 | Sutor winery Slovenia
Processing Record 9 of Set 1 | Cape Point Vineyards winery South Africa
Processing Record 10 of Set 1 | Streicker winery Australia
Processing Record 11 of Set 1 | Casa Tiene Vista winery US
Processing Record 12 of Set 1 | Heitz winery US
Processing Record 13 of Set 1 | Malene Wines winery US
Processing Record 14 of Set 1 | Pinhal da Torre winery Portugal
Processing Record 15 of Set 1 | La Confrérie de Oisly et Thésée winery France
Processing Record 16 of Set 1 | Marcel Martin winery France
Processing Record 17 of Set 1 | Año Verde wine

Processing Record 19 of Set 4 | Verse & Chorus winery US
Processing Record 20 of Set 4 | Clouds Rest winery US
Processing Record 21 of Set 4 | Kadiem winery US
Processing Record 22 of Set 4 | Red Stich winery US
Processing Record 23 of Set 4 | Lantz winery US
Processing Record 24 of Set 4 | Carte Blanche winery US
Processing Record 25 of Set 4 | Tenuta Poggio Rosso winery Italy
Processing Record 26 of Set 4 | Counter Punch winery US
Processing Record 27 of Set 4 | Cristobal 1492 winery Argentina
Processing Record 28 of Set 4 | Les Vignerons de la Méditerranée winery France
Processing Record 29 of Set 4 | Tenuta Santomè winery Italy
Processing Record 30 of Set 4 | Prestwood winery US
Processing Record 31 of Set 4 | Pizzato winery Brazil
Processing Record 32 of Set 4 | Big Vine winery US
Processing Record 33 of Set 4 | Flying Angels winery Chile
Processing Record 34 of Set 4 | Clean Slate winery Germany
Processing Record 35 of Set 4 | Archer Roose winery Chile
Processing Record 36 of Set

Processing Record 9 of Set 7 | Kapcsandy Family Winery winery US
Processing Record 10 of Set 7 | Roney winery US
Processing Record 11 of Set 7 | Argana winery Argentina
Processing Record 12 of Set 7 | Anonymous winery US
Processing Record 13 of Set 7 | Château les Bormettes winery France
Processing Record 14 of Set 7 | Streamside winery US
Processing Record 15 of Set 7 | Birgit Braunstein winery Austria
Processing Record 16 of Set 7 | Roy J. Maier winery US
Processing Record 17 of Set 7 | Château de Fountgraves winery France
Processing Record 18 of Set 7 | Domaine Bernard Moreau et Fils winery France
Processing Record 19 of Set 7 | Jacuzzi winery US
Processing Record 20 of Set 7 | Butterfly Kiss winery US
Processing Record 21 of Set 7 | Imagine Moore winery US
Processing Record 22 of Set 7 | Firelands winery US
Processing Record 23 of Set 7 | Shadybrook winery US
Processing Record 24 of Set 7 | Dalla Valle winery US
Processing Record 25 of Set 7 | Bosman Family Vineyards winery South A

Processing Record 50 of Set 9 | 75 Wine Co. winery US
Processing Record 1 of Set 10 | Torres winery Chile
Processing Record 2 of Set 10 | 3 Badge Beverage winery US
Processing Record 3 of Set 10 | Château Toutigeac winery France
Processing Record 4 of Set 10 | Chateau Molyneaux winery US
Processing Record 5 of Set 10 | French Rabbit winery France
Processing Record 6 of Set 10 | C&B; winery US
Processing Record 7 of Set 10 | Crayelle Cellars winery US
Processing Record 8 of Set 10 | Unti winery US
Processing Record 9 of Set 10 | Bejot Vins et Terroirs winery France
Processing Record 10 of Set 10 | Think Tank winery US
Processing Record 11 of Set 10 | 60 North winery US
Processing Record 12 of Set 10 | Giordano winery Italy
Processing Record 13 of Set 10 | Fount winery US
Processing Record 14 of Set 10 | Mockingbird winery US
Processing Record 15 of Set 10 | Warramate winery Australia
Processing Record 16 of Set 10 | Bennett Family winery US
Processing Record 17 of Set 10 | Witness Tree 

Processing Record 39 of Set 12 | YaYa winery Chile
Processing Record 40 of Set 12 | Domaine de la Feuillarde winery France
Processing Record 41 of Set 12 | Charles Joguet winery France
Processing Record 42 of Set 12 | Strehn winery Austria
Processing Record 43 of Set 12 | Cuatro Vacas Gordas winery Argentina
Processing Record 44 of Set 12 | Doña Catalina winery Chile
Processing Record 45 of Set 12 | Leo Steen Wines winery US
Processing Record 46 of Set 12 | Exton Park Vineyard winery England
Processing Record 47 of Set 12 | The Setting winery US
Processing Record 48 of Set 12 | Swingset winery US
Processing Record 49 of Set 12 | Canoas winery Argentina
Processing Record 50 of Set 12 | Wayne Gretzky Estates winery US
Processing Record 1 of Set 13 | Paredon winery US
Processing Record 2 of Set 13 | Robert Perroud winery France
Processing Record 3 of Set 13 | Artisan Uprising winery US
Processing Record 4 of Set 13 | Bucklin winery US
Processing Record 5 of Set 13 | Villa del Monte winery

Processing Record 26 of Set 15 | Three of Cups winery US
Processing Record 27 of Set 15 | Cave de Rauzan winery France
Completed 700 of 7994 address
Processing Record 28 of Set 15 | Château de Belcier winery France
Processing Record 29 of Set 15 | Château de Piote winery France
Processing Record 30 of Set 15 | Château Haut Bessac winery France
Processing Record 31 of Set 15 | Kathryn Hall Vineyards winery US
Processing Record 32 of Set 15 | Domaine Jean-Philippe Fichet winery France
Processing Record 33 of Set 15 | Domeniile Sahateni winery Romania
Processing Record 34 of Set 15 | Rumelia Wine Cellar winery Bulgaria
Processing Record 35 of Set 15 | Domaine Grand St Paul winery France
Processing Record 36 of Set 15 | Siren Song winery US
Processing Record 37 of Set 15 | Old Acre Wines winery US
Processing Record 38 of Set 15 | Loubud winery US
Processing Record 39 of Set 15 | Elements winery US
Processing Record 40 of Set 15 | Kettle Valley winery Canada
Processing Record 41 of Set 15 |

Processing Record 14 of Set 18 | Marchand & Burch winery Australia
Processing Record 15 of Set 18 | Château de Landué winery France
Processing Record 16 of Set 18 | Maple Ridge winery US
Processing Record 17 of Set 18 | Old House winery US
Processing Record 18 of Set 18 | Bully Hill winery US
Processing Record 19 of Set 18 | Kosta Browne winery US
Processing Record 20 of Set 18 | Ormonde winery South Africa
Processing Record 21 of Set 18 | Origin winery US
Processing Record 22 of Set 18 | Las Vertientes winery Argentina
Processing Record 23 of Set 18 | Château Val d'Arenc winery France
Processing Record 24 of Set 18 | Clifford Bay Estate winery New Zealand
Processing Record 25 of Set 18 | Acronym winery US
Processing Record 26 of Set 18 | Mohua winery New Zealand
Processing Record 27 of Set 18 | Groupe Clinet winery France
Processing Record 28 of Set 18 | Domaine Michel Lafarge winery France
Processing Record 29 of Set 18 | Château Couhins-Lurton winery France
Processing Record 30 of S

In [10]:
# combine CSVs
wine_weather_1 = pd.read_csv('winery_weather_v2.csv', low_memory=False)
wine_weather_2 = pd.read_csv('winery_weather_startatHess.csv', low_memory=False)
wine_weather_3 = pd.read_csv('winery_weather_startatPend.csv', low_memory=False)

combined_winery = pd.concat([wine_weather_1, wine_weather_2, wine_weather_3])
combined_winery.rename(columns={'Average Precipitation (mm)': 'Average Daily Precipitation (mm)'}, inplace=True)
combined_winery


,Unnamed: 0,Winery,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,0,St. Julian winery US,283.486181,1016.253874,73.468022,0.118022
1,1,Sweet Cheeks winery US,284.928462,1016.972170,75.501813,0.108269
2,2,Kirkland Signature winery US,288.217995,1017.277418,74.106346,0.129918
3,3,Louis M. Martini winery US,287.733599,1015.947830,74.851236,0.076868
4,4,Richard Böcking winery Germany,283.138077,1013.212940,78.442610,0.091264
...,...,...,...,...,...,...
911,911,OZV winery US,289.849176,1015.587692,60.538791,0.043049
912,912,Ferguson Crest winery US,292.240137,1016.506291,65.695192,0.137115
913,913,Carmel Hills Winery winery US,282.675440,1013.180549,56.617720,0.058187
914,914,Castello Bonomi winery Italy,287.199615,1011.606346,73.320110,0.106813


In [12]:
combined_winery.to_csv('winery_weather_updated.csv')

In [20]:
#bring in reviews data
#update link
reviewdata = '../wine_reviews_cleaned.csv'
reviews = pd.read_csv(reviewdata, index_col=0)
review_df = reviews.reset_index()
rev_df = review_df.drop(columns='index')
rev_df.set_index('winery', inplace=True)
rev_df

,country,points,variety,winery,winery_search
0,US,87,Riesling,St. Julian,St. Julian winery US
1,US,87,Pinot Noir,Sweet Cheeks,Sweet Cheeks winery US
2,US,87,Cabernet Sauvignon,Kirkland Signature,Kirkland Signature winery US
3,US,87,Cabernet Sauvignon,Louis M. Martini,Louis M. Martini winery US
4,US,87,Chardonnay,Mirassou,Mirassou winery US
...,...,...,...,...,...
55456,New Zealand,90,Chardonnay,Dog Point,Dog Point winery New Zealand
55457,Portugal,90,Pinot Noir,Caves Transmontanas,Caves Transmontanas winery Portugal
55458,Israel,90,Cabernet Sauvignon,Dalton,Dalton winery Israel
55459,Germany,90,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),Dr. H. Thanisch (Erben Müller-Burggraef) winer...


In [27]:
combined_winery.set_index('Winery', inplace=True)
combined_winery.drop(columns=['Unnamed: 0'], inplace=True)
combined_winery

,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
Winery,,,,
St. Julian winery US,283.486181,1016.253874,73.468022,0.118022
Sweet Cheeks winery US,284.928462,1016.972170,75.501813,0.108269
Kirkland Signature winery US,288.217995,1017.277418,74.106346,0.129918
Louis M. Martini winery US,287.733599,1015.947830,74.851236,0.076868
Richard Böcking winery Germany,283.138077,1013.212940,78.442610,0.091264
...,...,...,...,...
OZV winery US,289.849176,1015.587692,60.538791,0.043049
Ferguson Crest winery US,292.240137,1016.506291,65.695192,0.137115
Carmel Hills Winery winery US,282.675440,1013.180549,56.617720,0.058187


In [31]:
#join with lat long data
latlong_df.set_index('winery_search_term', inplace=True)
latlong_df.drop(columns=['Unnamed: 0'], inplace=True)
latlong_df

,latitude,longitude,number_of_results,status
winery_search_term,,,,
St. Julian winery US,42.212251,-85.891713,1,OK
Sweet Cheeks winery US,43.956670,-123.279135,1,OK
Kirkland Signature winery US,35.896713,-84.140546,1,OK
Louis M. Martini winery US,38.491708,-122.452308,1,OK
Richard Böcking winery Germany,49.947177,7.112136,1,OK
...,...,...,...,...
Ferguson Crest winery US,33.313613,-96.659946,1,OK
Carmel Hills Winery winery US,43.949742,-103.522590,1,OK
Castello Bonomi winery Italy,45.571648,9.967799,1,OK


In [35]:
combined_winery_full = combined_winery.join(latlong_df, on='Winery')
combined_winery_full.drop(columns=['number_of_results', 'status'], inplace=True)
combined_winery_full

,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm),latitude,longitude
Winery,,,,,,
St. Julian winery US,283.486181,1016.253874,73.468022,0.118022,42.212251,-85.891713
Sweet Cheeks winery US,284.928462,1016.972170,75.501813,0.108269,43.956670,-123.279135
Kirkland Signature winery US,288.217995,1017.277418,74.106346,0.129918,35.896713,-84.140546
Louis M. Martini winery US,287.733599,1015.947830,74.851236,0.076868,38.491708,-122.452308
Richard Böcking winery Germany,283.138077,1013.212940,78.442610,0.091264,49.947177,7.112136
...,...,...,...,...,...,...
OZV winery US,289.849176,1015.587692,60.538791,0.043049,38.136551,-121.241532
Ferguson Crest winery US,292.240137,1016.506291,65.695192,0.137115,33.313613,-96.659946
Carmel Hills Winery winery US,282.675440,1013.180549,56.617720,0.058187,43.949742,-103.522590


In [37]:
combined_winery_full = combined_winery_full[["latitude", "longitude", "Average Temperature (Kelvin)", "Average Air Pressure (hPa)", "Average Humidity (%)", "Average Daily Precipitation (mm)"]]
combined_winery_full

,latitude,longitude,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
Winery,,,,,,
St. Julian winery US,42.212251,-85.891713,283.486181,1016.253874,73.468022,0.118022
Sweet Cheeks winery US,43.956670,-123.279135,284.928462,1016.972170,75.501813,0.108269
Kirkland Signature winery US,35.896713,-84.140546,288.217995,1017.277418,74.106346,0.129918
Louis M. Martini winery US,38.491708,-122.452308,287.733599,1015.947830,74.851236,0.076868
Richard Böcking winery Germany,49.947177,7.112136,283.138077,1013.212940,78.442610,0.091264
...,...,...,...,...,...,...
OZV winery US,38.136551,-121.241532,289.849176,1015.587692,60.538791,0.043049
Ferguson Crest winery US,33.313613,-96.659946,292.240137,1016.506291,65.695192,0.137115
Carmel Hills Winery winery US,43.949742,-103.522590,282.675440,1013.180549,56.617720,0.058187


In [38]:
output = rev_df.join(combined_winery_full, on='winery_search')
output

,country,points,variety,winery_search,latitude,longitude,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
winery,,,,,,,,,,
St. Julian,US,87,Riesling,St. Julian winery US,42.212251,-85.891713,283.486181,1016.253874,73.468022,0.118022
Sweet Cheeks,US,87,Pinot Noir,Sweet Cheeks winery US,43.956670,-123.279135,284.928462,1016.972170,75.501813,0.108269
Kirkland Signature,US,87,Cabernet Sauvignon,Kirkland Signature winery US,35.896713,-84.140546,288.217995,1017.277418,74.106346,0.129918
Louis M. Martini,US,87,Cabernet Sauvignon,Louis M. Martini winery US,38.491708,-122.452308,287.733599,1015.947830,74.851236,0.076868
Mirassou,US,87,Chardonnay,Mirassou winery US,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Dog Point,New Zealand,90,Chardonnay,Dog Point winery New Zealand,-41.527502,173.847294,286.897115,1014.387500,78.211703,0.106566
Caves Transmontanas,Portugal,90,Pinot Noir,Caves Transmontanas winery Portugal,41.260544,-7.469930,289.790577,1016.270220,76.412170,0.095330
Dalton,Israel,90,Cabernet Sauvignon,Dalton winery Israel,33.029546,35.476761,293.535549,1012.010220,64.654615,0.054835


In [40]:
clean_df = output.dropna().reset_index()
clean_df

,winery,country,points,variety,winery_search,latitude,longitude,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,St. Julian,US,87,Riesling,St. Julian winery US,42.212251,-85.891713,283.486181,1016.253874,73.468022,0.118022
1,Sweet Cheeks,US,87,Pinot Noir,Sweet Cheeks winery US,43.956670,-123.279135,284.928462,1016.972170,75.501813,0.108269
2,Kirkland Signature,US,87,Cabernet Sauvignon,Kirkland Signature winery US,35.896713,-84.140546,288.217995,1017.277418,74.106346,0.129918
3,Louis M. Martini,US,87,Cabernet Sauvignon,Louis M. Martini winery US,38.491708,-122.452308,287.733599,1015.947830,74.851236,0.076868
4,Richard Böcking,Germany,87,Riesling,Richard Böcking winery Germany,49.947177,7.112136,283.138077,1013.212940,78.442610,0.091264
...,...,...,...,...,...,...,...,...,...,...,...
53082,Dog Point,New Zealand,90,Chardonnay,Dog Point winery New Zealand,-41.527502,173.847294,286.897115,1014.387500,78.211703,0.106566
53083,Caves Transmontanas,Portugal,90,Pinot Noir,Caves Transmontanas winery Portugal,41.260544,-7.469930,289.790577,1016.270220,76.412170,0.095330
53084,Dalton,Israel,90,Cabernet Sauvignon,Dalton winery Israel,33.029546,35.476761,293.535549,1012.010220,64.654615,0.054835
53085,Dr. H. Thanisch (Erben Müller-Burggraef),Germany,90,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef) winer...,49.905538,7.035938,283.137940,1013.212665,78.443571,0.091264


In [41]:
clean_df.to_csv('weather_scaled_updated.csv')

#### County weather pull 

In [42]:
county_data = 'onecit_zips.csv'
county_latlong_df = pd.read_csv(county_data, low_memory=False)
county_latlong_df

,zip,lat,lng,city,state_id,state_name,county_name
0,1001,42.06259,-72.62589,Agawam,MA,Massachusetts,Hampden
1,1002,42.37492,-72.46210,Amherst,MA,Massachusetts,Hampshire
2,1005,42.42017,-72.10615,Barre,MA,Massachusetts,Worcester
3,1029,42.19632,-73.04836,East Otis,MA,Massachusetts,Berkshire
4,1054,42.47363,-72.48746,Leverett,MA,Massachusetts,Franklin
...,...,...,...,...,...,...,...
1792,99017,47.18881,-117.87560,Lamont,WA,Washington,Whitman
1793,99118,48.89659,-118.63900,Curlew,WA,Washington,Ferry
1794,99119,48.50390,-117.28854,Cusick,WA,Washington,Pend Oreille
1795,99323,46.19237,-118.84285,Burbank,WA,Washington,Walla Walla


In [43]:
# Create lists of latitudes and longitudes
county_lats = list(county_latlong_df['lat'])
county_lngs = list(county_latlong_df['lng'])

county_zip = list(county_latlong_df['zip'])

In [44]:
# create function for county weather
def get_county_weather_data (county, county_count):
    
    # Create endpoint URL with each county
    county_url = url + "&lat=" + str(county_lats[county_count]) + "&lon=" + str(county_lngs[county_count])
    
    county_temp_data  = []
    county_pressure_data = []
    county_humidity_data =[]
    county_precip_data = []
    
    # Parse the JSON and retrieve data.
    county_weather = requests.get(county_url).json()
    
    for j in range (0, 364):
        temp = county_weather["result"][j]["temp"]["mean"]
        county_temp_data.append(temp)
        pressure = county_weather["result"][j]["pressure"]["mean"]
        county_pressure_data.append(pressure)
        humidity = county_weather["result"][j]["humidity"]["mean"]
        county_humidity_data.append(humidity)
        precip = county_weather["result"][j]["precipitation"]["mean"]
        county_precip_data.append(precip)
                
    # calculate averages
    avg_temp = sum(county_temp_data) / len(county_temp_data)
    avg_pressure = sum(county_pressure_data) / len(county_pressure_data)
    avg_humidity = sum(county_humidity_data) / len(county_humidity_data)
    avg_precip = sum(county_precip_data) / len(county_precip_data)
    
    return avg_temp, avg_pressure, avg_humidity, avg_precip

In [45]:

# Create an empty list to hold the weather data.
county_weather_data = []
output_filename = 'uscounty_weather_v2.csv'

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
county_count = 0

# Loop through all the wineries in the list.
for i in range (0, 1796):
    county = county_zip[i]
    
    # Group counties in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {county}")
    
    # Add 1 to the record count
    record_count += 1
    
    # Run an API request for each of the wineries.
    try:
        # Parse the JSON and retrieve data.
        avg_temp, avg_pressure, avg_humidity, avg_precip = get_county_weather_data(county, i)
               
        # Append the winery information into city_data list.
        county_weather_data.append({"zip": county_zip[county_count],
                          "Average Temperature (Kelvin)": avg_temp,
                          "Average Air Pressure (hPa)": avg_pressure,
                          "Average Humidity (%)": avg_humidity,
                          "Average Daily Precipitation (mm)": avg_precip})

    # If an error is experienced, skip the county.
    except:
        print("Weather not found. Skipping...")
        pass
    
    # Add to county count
    county_count += 1
    
    # Every 500 addresses, save progress to file(in case of a failure so we have something!)
    if len(county_weather_data) % 500 == 0:
        pd.DataFrame(county_weather_data).to_csv("{}_bak".format(output_filename))
        
    # save full results to CSV
    pd.DataFrame(county_weather_data).to_csv(output_filename)

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")



Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | 1001
Processing Record 2 of Set 1 | 1002
Processing Record 3 of Set 1 | 1005
Processing Record 4 of Set 1 | 1029
Processing Record 5 of Set 1 | 1054
Processing Record 6 of Set 1 | 1431
Processing Record 7 of Set 1 | 1810
Processing Record 8 of Set 1 | 2019
Processing Record 9 of Set 1 | 2043
Processing Record 10 of Set 1 | 2048
Processing Record 11 of Set 1 | 2108
Processing Record 12 of Set 1 | 2532
Processing Record 13 of Set 1 | 2535
Processing Record 14 of Set 1 | 2554
Processing Record 15 of Set 1 | 2802
Processing Record 16 of Set 1 | 2804
Processing Record 17 of Set 1 | 2816
Processing Record 18 of Set 1 | 2835
Processing Record 19 of Set 1 | 3031
Processing Record 20 of Set 1 | 3032
Processing Record 21 of Set 1 | 3046
Processing Record 22 of Set 1 | 3215
Processing Record 23 of Set 1 | 3218
Processing Record 24 of Set 1 | 3227
Processing Record 25 of Set 1 | 3280
Processing Record 26 of 

Processing Record 19 of Set 5 | 22980
Processing Record 20 of Set 5 | 23002
Processing Record 21 of Set 5 | 23004
Processing Record 22 of Set 5 | 23005
Processing Record 23 of Set 5 | 23009
Processing Record 24 of Set 5 | 23011
Processing Record 25 of Set 5 | 23021
Processing Record 26 of Set 5 | 23023
Processing Record 27 of Set 5 | 23030
Processing Record 28 of Set 5 | 23038
Processing Record 29 of Set 5 | 23059
Processing Record 30 of Set 5 | 23112
Processing Record 31 of Set 5 | 23139
Processing Record 32 of Set 5 | 23168
Processing Record 33 of Set 5 | 23187
Processing Record 34 of Set 5 | 23301
Processing Record 35 of Set 5 | 23304
Processing Record 36 of Set 5 | 23320
Processing Record 37 of Set 5 | 23451
Processing Record 38 of Set 5 | 23601
Processing Record 39 of Set 5 | 23651
Processing Record 40 of Set 5 | 23662
Processing Record 41 of Set 5 | 23701
Processing Record 42 of Set 5 | 23801
Processing Record 43 of Set 5 | 23803
Processing Record 44 of Set 5 | 23821
Processing R

Processing Record 36 of Set 9 | 29810
Processing Record 37 of Set 9 | 29812
Processing Record 38 of Set 9 | 29821
Processing Record 39 of Set 9 | 29824
Processing Record 40 of Set 9 | 29912
Processing Record 41 of Set 9 | 30002
Processing Record 42 of Set 9 | 30008
Processing Record 43 of Set 9 | 30011
Processing Record 44 of Set 9 | 30012
Processing Record 45 of Set 9 | 30014
Processing Record 46 of Set 9 | 30017
Processing Record 47 of Set 9 | 30025
Processing Record 48 of Set 9 | 30103
Processing Record 49 of Set 9 | 30110
Processing Record 50 of Set 9 | 30122
Processing Record 1 of Set 10 | 30132
Processing Record 2 of Set 10 | 30139
Processing Record 3 of Set 10 | 30204
Processing Record 4 of Set 10 | 30216
Processing Record 5 of Set 10 | 30217
Processing Record 6 of Set 10 | 30218
Processing Record 7 of Set 10 | 30220
Processing Record 8 of Set 10 | 30223
Processing Record 9 of Set 10 | 30230
Processing Record 10 of Set 10 | 30236
Processing Record 11 of Set 10 | 30285
Processing

Processing Record 48 of Set 13 | 38311
Processing Record 49 of Set 13 | 38326
Processing Record 50 of Set 13 | 38401
Processing Record 1 of Set 14 | 38504
Processing Record 2 of Set 14 | 38541
Processing Record 3 of Set 14 | 38549
Processing Record 4 of Set 14 | 38585
Processing Record 5 of Set 14 | 38606
Processing Record 6 of Set 14 | 38610
Processing Record 7 of Set 14 | 38614
Processing Record 8 of Set 14 | 38618
Processing Record 9 of Set 14 | 38622
Processing Record 10 of Set 14 | 38626
Processing Record 11 of Set 14 | 38702
Processing Record 12 of Set 14 | 38704
Processing Record 13 of Set 14 | 38721
Processing Record 14 of Set 14 | 38736
Processing Record 15 of Set 14 | 38824
Processing Record 16 of Set 14 | 38827
Processing Record 17 of Set 14 | 38834
Processing Record 18 of Set 14 | 38841
Processing Record 19 of Set 14 | 38843
Processing Record 20 of Set 14 | 38850
Processing Record 21 of Set 14 | 38901
Processing Record 22 of Set 14 | 38920
Processing Record 23 of Set 14 | 3

Processing Record 10 of Set 18 | 49611
Processing Record 11 of Set 18 | 49613
Processing Record 12 of Set 18 | 49616
Processing Record 13 of Set 18 | 49621
Processing Record 14 of Set 18 | 49632
Processing Record 15 of Set 18 | 49633
Processing Record 16 of Set 18 | 49637
Processing Record 17 of Set 18 | 49701
Processing Record 18 of Set 18 | 49705
Processing Record 19 of Set 18 | 49707
Processing Record 20 of Set 18 | 49709
Processing Record 21 of Set 18 | 49710
Processing Record 22 of Set 18 | 49712
Processing Record 23 of Set 18 | 49719
Processing Record 24 of Set 18 | 49743
Processing Record 25 of Set 18 | 49801
Processing Record 26 of Set 18 | 49805
Processing Record 27 of Set 18 | 49806
Processing Record 28 of Set 18 | 49807
Processing Record 29 of Set 18 | 49808
Processing Record 30 of Set 18 | 49812
Processing Record 31 of Set 18 | 49817
Processing Record 32 of Set 18 | 49853
Processing Record 33 of Set 18 | 49902
Processing Record 34 of Set 18 | 49905
Processing Record 35 of S

Processing Record 21 of Set 22 | 57717
Processing Record 22 of Set 22 | 57730
Processing Record 23 of Set 22 | 57735
Processing Record 24 of Set 22 | 58009
Processing Record 25 of Set 22 | 58013
Processing Record 26 of Set 22 | 58027
Processing Record 27 of Set 22 | 58031
Processing Record 28 of Set 22 | 58201
Processing Record 29 of Set 22 | 58210
Processing Record 30 of Set 22 | 58216
Processing Record 31 of Set 22 | 58239
Processing Record 32 of Set 22 | 58316
Processing Record 33 of Set 22 | 58317
Processing Record 34 of Set 22 | 58318
Processing Record 35 of Set 22 | 58332
Processing Record 36 of Set 22 | 58356
Processing Record 37 of Set 22 | 58401
Processing Record 38 of Set 22 | 58415
Processing Record 39 of Set 22 | 58416
Processing Record 40 of Set 22 | 58421
Processing Record 41 of Set 22 | 58428
Processing Record 42 of Set 22 | 58430
Processing Record 43 of Set 22 | 58436
Processing Record 44 of Set 22 | 58477
Processing Record 45 of Set 22 | 58520
Processing Record 46 of S

Processing Record 32 of Set 26 | 68636
Processing Record 33 of Set 26 | 68710
Processing Record 34 of Set 26 | 68753
Processing Record 35 of Set 26 | 68879
Processing Record 36 of Set 26 | 68922
Processing Record 37 of Set 26 | 68924
Processing Record 38 of Set 26 | 68937
Processing Record 39 of Set 26 | 68943
Processing Record 40 of Set 26 | 69001
Processing Record 41 of Set 26 | 69021
Processing Record 42 of Set 26 | 69024
Processing Record 43 of Set 26 | 69025
Processing Record 44 of Set 26 | 69032
Processing Record 45 of Set 26 | 69121
Processing Record 46 of Set 26 | 69125
Processing Record 47 of Set 26 | 69127
Processing Record 48 of Set 26 | 69128
Processing Record 49 of Set 26 | 69135
Processing Record 50 of Set 26 | 69147
Processing Record 1 of Set 27 | 69152
Processing Record 2 of Set 27 | 69301
Processing Record 3 of Set 27 | 69337
Processing Record 4 of Set 27 | 69341
Processing Record 5 of Set 27 | 69345
Processing Record 6 of Set 27 | 70032
Processing Record 7 of Set 27 |

Processing Record 43 of Set 30 | 78108
Processing Record 44 of Set 30 | 78113
Processing Record 45 of Set 30 | 78122
Processing Record 46 of Set 30 | 78330
Processing Record 47 of Set 30 | 78332
Processing Record 48 of Set 30 | 78336
Processing Record 49 of Set 30 | 78338
Processing Record 50 of Set 30 | 78358
Processing Record 1 of Set 31 | 78361
Processing Record 2 of Set 31 | 78363
Processing Record 3 of Set 31 | 78501
Processing Record 4 of Set 31 | 78536
Processing Record 5 of Set 31 | 78561
Processing Record 6 of Set 31 | 78602
Processing Record 7 of Set 31 | 78605
Processing Record 8 of Set 31 | 78606
Processing Record 9 of Set 31 | 78610
Processing Record 10 of Set 31 | 78617
Processing Record 11 of Set 31 | 78618
Processing Record 12 of Set 31 | 78801
Processing Record 13 of Set 31 | 78827
Processing Record 14 of Set 31 | 78829
Processing Record 15 of Set 31 | 78832
Processing Record 16 of Set 31 | 78833
Processing Record 17 of Set 31 | 78837
Processing Record 18 of Set 31 | 7

Processing Record 5 of Set 35 | 89316
Processing Record 6 of Set 35 | 89402
Processing Record 7 of Set 35 | 89406
Processing Record 8 of Set 35 | 89418
Processing Record 9 of Set 35 | 89440
Processing Record 10 of Set 35 | 89701
Processing Record 11 of Set 35 | 89801
Processing Record 12 of Set 35 | 90001
Processing Record 13 of Set 35 | 91320
Processing Record 14 of Set 35 | 91701
Processing Record 15 of Set 35 | 91752
Processing Record 16 of Set 35 | 91901
Processing Record 17 of Set 35 | 92222
Processing Record 18 of Set 35 | 92328
Processing Record 19 of Set 35 | 93013
Processing Record 20 of Set 35 | 93201
Processing Record 21 of Set 35 | 93203
Processing Record 22 of Set 35 | 93210
Processing Record 23 of Set 35 | 93401
Processing Record 24 of Set 35 | 93426
Processing Record 25 of Set 35 | 93512
Processing Record 26 of Set 35 | 93601
Processing Record 27 of Set 35 | 93620
Processing Record 28 of Set 35 | 93623
Processing Record 29 of Set 35 | 94002
Processing Record 30 of Set 35

In [46]:
county_weather_data_df = pd.DataFrame(county_weather_data)

county_weather_df = pd.merge(county_latlong_df, county_weather_data_df, on = "zip", how = "left")

,zip,lat,lng,city,state_id,state_name,county_name,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,1001,42.06259,-72.62589,Agawam,MA,Massachusetts,Hampden,283.257967,1016.129560,70.273709,0.126154
1,1002,42.37492,-72.46210,Amherst,MA,Massachusetts,Hampshire,283.257967,1016.129560,70.273709,0.126154
2,1005,42.42017,-72.10615,Barre,MA,Massachusetts,Worcester,283.368324,1015.384533,69.594835,0.135027
3,1029,42.19632,-73.04836,East Otis,MA,Massachusetts,Berkshire,283.257967,1016.129560,70.273709,0.126154
4,1054,42.47363,-72.48746,Leverett,MA,Massachusetts,Franklin,283.257967,1016.129560,70.273709,0.126154
...,...,...,...,...,...,...,...,...,...,...,...
1792,99017,47.18881,-117.87560,Lamont,WA,Washington,Whitman,282.530687,1014.477198,63.684863,0.064533
1793,99118,48.89659,-118.63900,Curlew,WA,Washington,Ferry,281.905742,1008.299066,68.823764,0.044890
1794,99119,48.50390,-117.28854,Cusick,WA,Washington,Pend Oreille,282.530687,1014.477198,63.684863,0.064533
1795,99323,46.19237,-118.84285,Burbank,WA,Washington,Walla Walla,282.688049,999.032280,60.418104,0.034863


In [47]:
pd.DataFrame(county_weather_df).to_csv(output_filename)